# Inferencia en imágenes

In [14]:
from datasets import load_from_disk
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
import numpy as np
from Utils import confusion_matrix

In [15]:
# Parámetros
_checkpoint = 'SavedModels/Cal_ViT-large-patch16-224_A.ipynb/checkpoint-3780'  # path al checkpoint a cargar
classifier = pipeline("image-classification", model=_checkpoint, device="mps")

## En dataset

In [16]:
_dataset_test = '/Users/julio/Documentos-Local/data/VinDr-Mammo/subsets/calcifications_3Categories_split/test'

dataset = load_from_disk(_dataset_test)

In [17]:
labels = dataset.features['label'].names
label2id = {c:idx for idx,c in enumerate(labels)}
id2label = {idx:c for idx,c in enumerate(labels)}

### Mostrar algunas predicciones

In [ ]:
cont = 0
for i, out in enumerate(classifier(KeyDataset(dataset, "image"))):
    if cont > 100:
        break
    else:
        predicted_label = out[0]['label']
        predicted_score = out[0]['score']
        real_label = id2label[dataset[i]['label']]  # Obtener la etiqueta real
        print(f"Predicción: {predicted_label} - {predicted_score * 100:.2f}%, Etiqueta real: {real_label}")
        cont += 1


### Matriz de confusion

In [ ]:
# Inicializar listas para las etiquetas reales y predichas
y_true = []
y_pred = []

# Inferencia sobre el conjunto de test
for sample in dataset:
    # Realizar predicción sobre la imagen
    prediction = classifier(sample['image'])[0]  # Primera predicción más probable
    predicted_label = prediction['label']
    
    # Guardar las etiquetas real y predicha
    y_true.append(sample['label'])  # Índice de etiqueta real
    y_pred.append(label2id[predicted_label])  # Convertir la predicción a índice

# Crear un diccionario para simular la estructura esperada por tu función
samples_dict = {'label': np.array(y_true)}  # Las etiquetas reales como array

# Llamar a la función de matriz de confusión sin modificarla
confusion_matrix(samples_dict, y_pred, labels)

## Por imagen 

In [3]:
classifier('/Users/julio/Library/Mobile Documents/com~apple~CloudDocs/Downloads/output.png')

[{'label': 'calcificaciones', 'score': 0.9976891279220581},
 {'label': 'masas', 'score': 0.14085496962070465},
 {'label': 'no_encontrado', 'score': 0.0005524149746634066}]